# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW2-bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW2-binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'KJ7432.9.A.QJT86'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PASS", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1S', 'insta_score': 0.8339, 'expected_score': 78, 'adjustment': 42},
 {'call': '2S', 'insta_score': 0.1631, 'expected_score': 51, 'adjustment': 8}]

In [4]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.993}]

In [5]:
bid.samples

['KJxxxx.9.A.QJT8x x.AKJ8xxxx.xx.A9 Qx.Txx.K8x.Kxxxx AT98.Q.QJT9xxx.x 0.39012',
 'KJxxxx.9.A.QJT8x AQ.AQJTxxx.J8x.x 8.K8xx.Txx.Kxxxx T9xx.x.KQ9xxx.A9 0.36071',
 'KJxxxx.9.A.QJT8x 8x.AKJTxxx.x.AKx A.8xx.T9xxx.9xxx QT9x.Qx.KQJ8xx.x 0.31275',
 'KJxxxx.9.A.QJT8x xx.AQJ8xx.Kx.Axx 9.Kxxx.9xxx.Kxxx AQT8.Tx.QJT8xx.9 0.30764',
 'KJxxxx.9.A.QJT8x 9x.AKQxxxx.xx.Ax T.Txx.QJxx.K9xxx AQ8x.J8.KT98xx.x 0.27324',
 'KJxxxx.9.A.QJT8x Q8.AQJT8xxx.Kx.K x.Kxxx.JTxx.xxxx AT9x..Q98xxx.A9x 0.25882',
 'KJxxxx.9.A.QJT8x Q8.KQJT8xx.T9.Ax 9.Axx.Jxxx.K9xxx ATxx.xx.KQ8xxx.x 0.25252',
 'KJxxxx.9.A.QJT8x x.AKQ8xx.Q9x.Kxx QT.JTxx.8xx.Axxx A98x.xx.KJTxxx.9 0.24375',
 'KJxxxx.9.A.QJT8x QT.AT8xxxx.x.AKx 8.KQJxx.QT9xx.xx A9xx..KJ8xxx.9xx 0.19063',
 'KJxxxx.9.A.QJT8x 98x.AKQJxx.x.K9x Qx.Txxx.KQx.Axxx AT.8x.JT98xxxx.x 0.18854']

In [6]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D", "PASS", "3N" , "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.9984,
  'expected_score': -293,
  'adjustment': 50},
 {'call': 'X',
  'insta_score': 0.0012,
  'expected_score': -375,
  'adjustment': -400}]

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

KJxxxx.9.A.QJT8x T8xx.AK8.T9xxx.K 9.QTxx.KQJx.A9xx AQ.Jxxxx.8xx.xxx
KJxxxx.9.A.QJT8x Q8x.QJxx.KJ9x.Kx T.AKT8.T8xx.Axxx A9x.xxxx.Qxxx.9x
KJxxxx.9.A.QJT8x AQT.JTxx.xxxxx.K x.AQ8x.KQT9.9xxx 98x.Kxxx.J8x.Axx
KJxxxx.9.A.QJT8x Axx.Jxx.J98x.AK9 T.Axxx.KQxxx.xxx Q98.KQT8x.Txx.xx
KJxxxx.9.A.QJT8x AQxx.Kxxx.QT9x.x 8.QJ8x.KJ8xx.A9x T9.ATxx.xxx.Kxxx
KJxxxx.9.A.QJT8x AT8.QTxx.QJxx.Kx 9.AJxx.K98xx.Axx Qxx.K8xx.Txx.9xx
KJxxxx.9.A.QJT8x QT.Qxxxx.QJ9x.A9 9.AKJx.8xxxx.Kxx A8xx.T8x.KTx.xxx
KJxxxx.9.A.QJT8x 98x.AJ8xx.xxx.A9 x.KTxx.KJ98xx.Kx AQT.Qxx.QTx.xxxx
KJxxxx.9.A.QJT8x AT9x.KTxx.KQx.9x x.Axxx.T9xxxx.AK Q8.QJ8x.J8x.xxxx
KJxxxx.9.A.QJT8x xx.AJxx.Txx.A9xx 9.KTxx.KQ9xxx.Kx AQT8.Q8xx.J8x.xx
KJxxxx.9.A.QJT8x AT9x.KTxx.JT9x.x x.QJ8x.K8xxx.AK9 Q8.Axxx.Qxx.xxxx
KJxxxx.9.A.QJT8x T8x.xxx.Qxxx.AKx x.AQxx.KJT98.9xx AQ9.KJT8x.xxx.xx
KJxxxx.9.A.QJT8x A9x.ATxx.Qxx.xxx T.K8xx.KJ8xxx.K9 Q8x.QJxx.T9x.Axx
KJxxxx.9.A.QJT8x A8x.Kxxxx.JTx.Ax 9.AJ8x.KQ9xxx.Kx QTx.QTx.8xx.9xxx
KJxxxx.9.A.QJT8x T9x.Txxx.QJx.AKx x.AKJx.KTxxxx.